In [9]:
import os
import myServices 
import randomForestRegressor
import dask
import dask_cudf
from cuml import RandomForestClassifier as cuRF
from cuml.metrics import accuracy_score
import numpy as np
import pandas as pd

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))